# 크롤링으로 위경도 가져오기
* 따릉이 대여소 정보 가져오기
* 크롤링 위치 : https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo=1


In [39]:
# 라이브러리 로드
# requests는 작은 웹브라우저로 웹사이트 내용을 가져옵니다.
import requests

# bs4에서 BeautifulSoup 을 bs 라는 별칭으로 읽어옵니다. 웹페이지를 파싱합니다.
from bs4 import BeautifulSoup as bs

# random은 랜덤한 시간차를 두고 가져오기 위해 사용합니다.
import random 

# time 으로 시간을 구합니다.
import time

# 크롤링 후 결과를 데이터프레임 형태로 보기 위해 판다스를 불러옵니다.
import pandas as pd

## 전체 목록 받아오기

1. requests.get 으로 url을 읽어옵니다.
1. response.status_code != 200 HTTP 응답값을 확인합니다.
1. bs(response.text, 'html.parser') 로 파싱합니다.
1. 복사해온 html 엘리먼트 값을 사용해서 크롤링하고자 하는 태그를 찾습니다.
1. not tbody 로 tobody 값이 없다면 종료하도록 합니다.
1. www.bikeseoul.com 사이트의 서버에 부담을 덜 주기 위해 시간차를 두고 가져옵니다.
time.sleep(random.uniform(1,3))
1. 크롤링 해서 가져온 대여소 정보를 리스트에 담아 줍니다.
1. 몇 페이지를 크롤링 하고 있는지 찍어봅니다. 
1. 페이지 하나를 불러왔다면 페이지 번호를 넣어주면 해당 페이지를 불러오는 함수를 만듭니다.
1. 이전 결과가 있다면 페이지번호를 하나씩 더해서 재귀호출을 합니다.
1. 같은 함수를 재귀호출해서 실행합니다.
1. 함수 내부에 response의 tbody 값을 전달해 각 대여소의 상세 정보를 받아오는 함수를 작성합니다.

In [40]:
# 크롤링 할 사이트
base_url = 'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

In [41]:
# pd.read_html(base_url+"1")
table = pd.read_html(f"{base_url}1")
table[0]

,대여소 이름,상태정보,거치대수,대여가능거치대수,위치
0,1001. 광진교 남단 사거리(디지털프라자앞),운영중,15,2,서울특별시 강동구 올림픽로 674 로석빌딩
1,1002. 해공공원(천호동),운영중,10,1,서울특별시 강동구 올림픽로 702 해공도서관
2,1003. 해공도서관앞,운영중,20,0,서울특별시 강동구 올림픽로 702 해공도서관
3,1004. 삼성광나루아파트 버스정류장,운영중,10,1,서울특별시 강동구 올림픽로 812
4,1006. 롯데캐슬 115동앞,운영중,15,0,서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트


## 각 대여소의 정보를 받아오기
1. tbody 값이 있는지 확인합니다.
1. tr을 반복문으로 순회합니다.
1. '.' 이 대여소명에 들어가는지 확인하고 이 문자로 대여소 번호와 대여소명을 가져옵니다. 
1. get_text()를 통해 문자만 가져올 수 있습니다.
1. 없다면 대여소번호를 np.nan 으로 결측치를 넣어주고 대여소는 텍스트 데이터를 넣어줍니다.
1. 거치대수를 가져옵니다.
1. 운영여부를 가져옵니다.
1. 주소를 가져옵니다.
1. 위도, 경도를 가져옵니다 .find('a')['param-data'] 로 찾습니다.
1. 위도, 경도를 "," 문자로 분리합니다.
1. 대여소 리스트에 info 리스트를 담습니다.
1. rent_stations 에 담고 다 만들었다면 반환합니다.

In [42]:
base_url

'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

In [43]:
response = requests.get(f"{base_url}1")

In [44]:
type(response.text)

str

In [45]:
html = bs(response.text)
type(html)

bs4.BeautifulSoup

In [46]:
# # => id
# . => class
#container > table > tbody > tr:nth-child(1) > td:nth-child(1) > a

trs = html.select("#container > table > tbody > tr")
len(trs)

5

In [47]:
# 대여소명
trs[0].select("td")[0].get_text()

'1001. 광진교 남단 사거리(디지털프라자앞)'

In [48]:
trs[0].select("td")[0].select("a")[0]["param-data"].split(",")

['37.54179382', '127.12474823']

In [49]:
#fist_thisBestseller_img
#bestseller_list > li:nth-child(1) > dl > dt.thumb

In [50]:
trs[0]

<tr>
<td class="pl10"><a href="#" param-data="37.54179382,127.12474823">1001. 광진교 남단 사거리(디지털프라자앞)</a></td>
<td class="pl10">
                        운영중</td>
<td class="tr">15</td>
<td class="tr">2</td>
<td class="tbleft mhid"><span>서울특별시 강동구 올림픽로 674 로석빌딩 </span></td>
</tr>

In [51]:
trs[0].select("td")[1].get_text()

'\r\n                        운영중'

In [52]:
trs[0].select("td")[0].get_text(strip=True)

'1001. 광진교 남단 사거리(디지털프라자앞)'

In [53]:
#  info = []
        # 대여소번호, 대여소명
        # 거치대수
        # 운영여부
        # 주소
        # 위도, 경도
def parse_tr(tr):
    info = []
    
    title = tr.select("td")[0].get_text()
    lat = tr.select("td")[0].select("a")[0]["param-data"].split(",")[0]
    long = tr.select("td")[0].select("a")[0]["param-data"].split(",")[1]
    onoff = tr.select("td")[1].get_text(strip=True)
    address = tr.select("td")[-1].get_text(strip=True)
    
    
    info.append(title)
    info.append(lat)
    info.append(long)
    info.append(onoff)
    info.append(address)
    
    return info

In [54]:
parse_tr(trs[0])

['1001. 광진교 남단 사거리(디지털프라자앞)',
 '37.54179382',
 '127.12474823',
 '운영중',
 '서울특별시 강동구 올림픽로 674 로석빌딩']

In [55]:
# ['대여소번호', '대여소', '거치대수', '상태', '주소', '위도', '경도'] 를 반환합니다.
def station_info(trs):
    rent_stations = []
    # tr을 반복문으로 순회합니다.
        # 대여소 정보를 담아준다.
    for tr in trs:
        info = parse_tr(tr)
        rent_stations.append(info)

    return rent_stations

In [56]:
# station_info
trs = html.select("#container > table > tbody > tr")
station_info(trs)

[['1001. 광진교 남단 사거리(디지털프라자앞)',
  '37.54179382',
  '127.12474823',
  '운영중',
  '서울특별시 강동구 올림픽로 674 로석빌딩'],
 ['1002. 해공공원(천호동)',
  '37.54526138',
  '127.12594604',
  '운영중',
  '서울특별시 강동구 올림픽로 702 해공도서관'],
 ['1003. 해공도서관앞',
  '37.54395676',
  '127.12548828',
  '운영중',
  '서울특별시 강동구 올림픽로 702 해공도서관'],
 ['1004. 삼성광나루아파트 버스정류장',
  '37.55332947',
  '127.12886810',
  '운영중',
  '서울특별시 강동구 올림픽로 812'],
 ['1006. 롯데캐슬 115동앞',
  '37.55486679',
  '127.14279938',
  '운영중',
  '서울특별시 강동구 고덕로 131 강동 롯데캐슬퍼스트아파트']]

In [57]:
base_url

'https://www.bikeseoul.com/app/station/moveStationSearchView.do?currentPageNo='

In [58]:
import numpy as np
random_time = np.random.uniform(0, 1)
random_time

0.7649508136655238

In [59]:
# 전체 대여소 정보를 담아줄 비어있는 리스트를 만듭니다.
all_stations = []

def crawling_rent_station(pnum):
    # random_time 만큼 쉬었다가 크롤링합니다.
    time.sleep(random_time)
    
    url = f"{base_url}{pnum}"
    response = requests.get(url)
    
    if response.status_code == 200:
        html = bs(response.text)
        trs = html.select("#container > table > tbody > tr")
        return trs
    else:
        return False

In [60]:
requests.get(f"{base_url}10000")

<Response [200]>

In [61]:
# 재귀호출을 사용하는 방법
all_station = []
def crawling_all_station(pnum):
    trs = crawling_rent_station(pnum)
    if trs :
        stations = station_info(trs)
        all_station.extend(stations)
        # 결과가 정상동작한다면 다음 페이지 번호를 호출합니다.
        pnum = pnum + 1
        # 재귀호출 => 자기 자신을 호출합니다.
        return crawling_all_station(pnum)
    else:
        # 페이지가 없다면 all_station 을 반환합니다.
        return all_station

In [62]:
from tqdm import tqdm, trange
# for문을 사용하는 방법
start_page_no = 1
end_page_no = 10

# 초기화
all_station = []
for pnum in trange(start_page_no, end_page_no+1):
    trs = crawling_rent_station(pnum)
    if trs :
        stations = station_info(trs)
        all_station.extend(stations)
        all_station

100%|██████████| 10/10 [00:14<00:00,  1.47s/it]


In [63]:
# 재귀호출
all_stations = crawling_all_station(406)

In [64]:
df = pd.DataFrame(data)
df.head()

NameError: name 'data' is not defined

## 함수 동작확인

In [65]:
# 전체를 돌려보기 전에 일부만 돌려본다.
crawling_rent_station(306)

[<tr>
 <td class="pl10"><a href="#" param-data="37.56597900,127.04311371">3553.마장역3번출구</a></td>
 <td class="pl10">
                         운영중</td>
 <td class="tr">1</td>
 <td class="tr">0</td>
 <td class="tbleft mhid"><span>서울특별시 성동구 마장로 302 </span></td>
 </tr>,
 <tr>
 <td class="pl10"><a href="#" param-data="37.56681824,127.02915192">3555.양지사거리(센트라스APT 115동앞)</a></td>
 <td class="pl10">
                         운영중</td>
 <td class="tr">1</td>
 <td class="tr">0</td>
 <td class="tbleft mhid"><span>서울특별시 성동구 왕십리로 410 센트라스 </span></td>
 </tr>,
 <tr>
 <td class="pl10"><a href="#" param-data="37.55610657,127.01978302">3556.신금호파크자이APT앞</a></td>
 <td class="pl10">
                         운영중</td>
 <td class="tr">1</td>
 <td class="tr">0</td>
 <td class="tbleft mhid"><span>서울특별시 성동구 금호동2가 1-5 </span></td>
 </tr>,
 <tr>
 <td class="pl10"><a href="#" param-data="37.55653000,127.03913116">3557.서울숲리버뷰자이APT105동 앞</a></td>
 <td class="pl10">
                         운영중</td>
 <td class="tr">1</td

## 데이터 프레임으로 일부 결과 저장 확인하기

In [68]:
header = ['대여소', '위도', '경도','상태', '주소']
df = pd.DataFrame(all_stations,  columns = header)
df

60

## 전체 크롤링 하기

In [ ]:
# 전체 코드를 돌리기 전에 다시 리스트를 비워줍니다.
all_stations = []
# 아래의 pnum에 1을 입력하면 1페이지부터 끝까지 가져옵니다.
# pnum에 306 을 306페이지부터 끝까지 가져옵니다.
pnum = 306
# pnum = 300
# 크롤링 함수를 호출합니다.
crawling_all_station(pnum)

In [ ]:
# 크롤링한 전체 결과를 프린트 한다.
all_stations[:2]

## 데이터 프레임 만들기

In [ ]:
header = ['대여소번호', '대여소', '거치대수', '상태', '주소', '위도', '경도']


In [ ]:
# 미리보기 합니다.


## CSV 파일로 저장하고 확인하기

In [ ]:
# csv 파일로 저장하기
df.to_csv("seoul_bike_station.csv", index=False)

In [ ]:
# 파일이 제대로 생성되었는지 확인
pd.read_csv("seoul_bike_station.csv").head()